<div align="center" dir="rtl">
<h2>تمرین چهارم درس بازیابی پیشرفته‌ی اطلاعات</h2>
<h3>گروه سی‌و‌سوم: اسامی</h3>
</div>
<div align="right" dir="rtl">
  <br>
    در این تمرین، تسک دسته‌بندی روی داده‌های گرد‌آوری شده از بوستان سعدی انجام می‌شود. ابتدا دیتاست مورد نیازمان از روی داده‌های خام که برای تمرین سوم جمع شده بود ساخته می‌شود، سپس قسمت پیش‌پردازش قرار دارد و در ادامه تسک روی داده‌ها انجام می‌شود.
</div>

<div align="right" dir="rtl">
    <h3>قسمت اول: آماده‌سازی دیتاست</h3>
  <br>
    در این قسمت فایل‌های داده‌های خام خوانده شده و دیتاست بیت به بیت برای کلاس‌بندی ایجاد می‌شود.
</div>

In [15]:
#import libraries for dataset generation
import pandas as pd
from os import listdir
!pip install hazm
!pip install torchtext
from hazm import *
import torch
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import itertools

You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  Using cached torchtext-0.12.0-cp39-cp39-win_amd64.whl (1.5 MB)


You should consider upgrading via the 'C:\Users\ASUS\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
all_files = listdir('Boostan/')

i = 0
df = pd.DataFrame(columns=['beyt', 'baab'])

for p in all_files:
    f = open('Boostan/' + p, 'r', encoding='utf8')
    
    has_poems = True
    
    while has_poems:
        poem = ''
        
        while True:
            s = f.readline()
            if '$' in s:
                has_poems = False
                break
            elif '_' in s:
                break
            
            #age injaiim yani s ye mesra boode
            beyt = s + f.readline()[:-1]
            df.loc[i] = [beyt, p[:-4]]
            i = i + 1

In [16]:
print(df.at[2000, 'beyt'])
print(df.at[2000, 'baab'])
print(df.shape[0])

نپنداری قول معقول چو قانع شد سیم سنگ
باب ششم در قناعت
3845


<div align="right" dir="rtl">
    <h3>قسمت دوم: پیش‌پردازش</h3>
  <br>
    در این قسمت یک نمونه‌ی پیش‌پردازش شده از دیتاست تهیه می‌شود. دیتاست پیش‌پردازش‌نشده هم در کنار آن حفظ می‌شود که امکان مقایسه‌ی نتیجه روی دو دیتاست وجود داشته باشد.
    توضیحات دقیق این که هر گام از پیش‌پردازش شامل چه مواردی است، در تمرین سوم آورده شده است.
</div>

In [17]:
#import libraries for preprocessing
from __future__ import unicode_literals
from hazm import *
import json
import codecs

In [18]:
df_raw = df.copy()

<div align="right" dir="rtl">
    <h4>مرحله‌ی اول پیش‌پردازش</h4>
  <br>
    در این گام، برخی موارد در واژگان (مانند «گه» که همان «گاه» است) اصلاح می‌شوند. عملیات نرمالایز کردن و لمتایز کردن هم انجام داده می‌شود.
</div>

In [19]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()

def preprocess_1(beyt):
    beyt = normalizer.normalize(beyt).replace(' ز ', ' از ').replace(' مر ', ' ') #normalize beshe va z beshe az
    
    words = word_tokenize(beyt)
    
    for j in range(0, len(words)):
        
        word = words[j]
        past_root = lemmatizer.lemmatize(word).split('#', 1)[0]
        past_root = past_root.replace('آ', 'ا')
        word_modified = word.replace(past_root, '')
        
        if word_modified != '':
            if word_modified[0] == 'ب' and len(word_modified) < 4:
                #yani fele maazi bode be forme ghadimi
                beyt = beyt.replace(word, word[1:])
        
        if word.endswith('گه'):
            beyt = beyt.replace(word, word[:-2]+'گاه')
        
        return beyt

for i in range(0, df.shape[0]):
    df.at[i, 'beyt'] = preprocess_1(df.at[i, 'beyt'])

<div align="right" dir="rtl">
    <h4>مرحله‌ی دوم پیش‌پردازش</h4>
  <br>
    در این گام، حالات باستانی افعال به حالت عادی تبدیل می‌شود تا افعال یک دست باشند.
</div>

In [20]:
def preprocess_2(beyt):
    words = word_tokenize(beyt)
    
    for j in range(0, len(words)):
        
        word = words[j]
        
        if word[0] == 'م':
            word_modified = 'ب' + word[1:]
            present_root = lemmatizer.lemmatize(word_modified).split('#', 1)[-1]
            present_root = present_root.replace('آ', 'ا')
            word_modified = word_modified.replace(present_root, '')
            
            if word_modified == 'ب':
                beyt = beyt.replace(word, 'ن'+word[1:])
    return beyt

for i in range(0, df.shape[0]):            
    df.at[i, 'beyt'] = preprocess_2(df.at[i, 'beyt'])

<div align="right" dir="rtl">
    <h4>مرحله‌ی سوم پیش‌پردازش</h4>
  <br>
    در این مرحله، استاپ ووردها حذف شده و برخی واژگان خاص پیش‌پردازش می‌شوند.
</div>

In [21]:
with open('Exception.txt', encoding='utf8') as exc:
    data = exc.read()
special_verbs = json.loads(data)

df_temp = pd.DataFrame(columns=['beyt', 'baab'])
stopwords = ['!','،','؟','ز','ار']+[normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]


def preprocess_3(beyt):
    words = word_tokenize(beyt)
    words = [t for t in words if t not in stopwords]
    lemmatized_tokens = []
    
    for t in words:
        if t in special_verbs.keys():
            lemmatized_tokens.append(t)
        else:
            lemmatized_tokens.append(lemmatizer.lemmatize(t).split('#')[0])
    
    lemmatized = ' '.join(lemmatized_tokens)
            
    return lemmatized, lemmatized_tokens

for i in range(0, df.shape[0]):
    pp = preprocess_3(df.at[i, 'beyt'])
            
    df.at[i, 'beyt'] = pp[0]
    df_temp.at[i, 'beyt'] = pp[1]


In [22]:
print(df.at[2000, 'beyt'])
print(df.at[2000, 'baab'])

نپنداری قول معقول چو قانع سیم سنگ
باب ششم در قناعت


In [23]:
#test preprocessing
preprocess_3(preprocess_2(preprocess_1('''نپنداری این قول معقول نیست
چو قانع شدی سیم و سنگت یکی است''')))[0]

'نپنداری قول معقول چو قانع شد سیم سنگ'

<div align="right" dir="rtl">
    <h4>تابع پیش‌پردازش تک‌بیت</h4>
  <br>
    حال یک تابع درست می‌کنیم که یک بیت را پیش‌پردازش کند تا بعدا هنگام دریافت ورودی سامانه از آن استفاده کنیم.
</div>

In [24]:
def preprocess(beyt):
    return preprocess_3(preprocess_2(preprocess_1(beyt)))[0]

In [25]:
#test preprocessing functions
print(df_raw.at[2000, 'beyt'] + '\n----')
print(df.at[2000, 'beyt'] + '\n----')
print(preprocess(df_raw.at[2000, 'beyt']))

نپنداری قول معقول چو قانع شد سیم سنگ
----
نپنداری قول معقول چو قانع سیم سنگ
----
نپنداری قول معقول چو قانع سیم سنگ


<div align="right" dir="rtl">
    <h3>قسمت سوم: کلاس‌بندی با scikit</h3>
  <br>
    در این قسمت با ؟؟؟ کلاس‌بندی می‌کنیم
</div>

In [26]:
#import libraries for classification (1)
import os
import re
import shutil
import string
from collections import Counter
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=None, norm='l2')
v = vectorizer.fit_transform(['نپنداری قول معقول چو قانع شد سیم سنگ', 'ممد ممد', 'ممد'])
#v = augment(v)

<div align=center>
		    <font size=4 color=green>
			    <br />
                مدل اول:
                </font>
                <font size=3>
                {اسم مدلتون رو اینجا بنویسید}
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

<div align=center>
		    <font size=4 color=green>
			    <br />
                مدل دوم:
                </font>
                <font size=3>
                {اسم مدلتون رو اینجا بنویسید}
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

<div align=center>
		    <font size=4 color=green>
			    <br />
                مدل سوم:
                </font>
                <font size=3>
                مدل شبکه عصبی استفاده از کتابخانه
                
torchtext
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>